## Dataset Preprocessing Notebook

#### Load request modules

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import os
from urllib.parse import urlparse
from os.path import basename
from rich.progress import track
import json

In [4]:
# initialize variables
base_dir = '../data'
hotspot_dir = f'{base_dir}/hotspot'

# download chuck size
chunk_size = 10_240

#  weather data files
urls = ['https://pub-6924d15baafa48c6a31c2d212b094174.r2.dev/environment/air_quality-13t20_97t106_20240910_20241211.csv.bz2',
        'https://pub-6924d15baafa48c6a31c2d212b094174.r2.dev/environment/weather-13t20_97t106_20240910_20241211.csv.bz2']

#### Load weather and air quality data

In [6]:
# download file from cloud storage
for url in urls:
    file = basename(url)
    parse_url = urlparse(url)
    print(f'Getting file {file}')
    with requests.get(url, stream=True) as res:
        if res.ok:
            size = int(res.headers.get('Content-Length'))
            with open(f'{base_dir}{parse_url.path}', 'wb') as f:
                for chuck in track(res.iter_content(chunk_size), total=size/chunk_size, description='Download...'):
                    f.write(chuck)
        else:
            print(f'Error getting file!!!')
print('Done...')

Getting file air_quality-13t20_97t106_20240910_20241211.csv.bz2


Output()

Getting file weather-13t20_97t106_20240910_20241211.csv.bz2


Output()

Done...


In [7]:
weather_df = pd.read_csv('../data/environment/weather-13t20_97t106_20240910_20241211.csv.bz2')
air_quality_df = pd.read_csv('../data/environment/air_quality-13t20_97t106_20240910_20241211.csv.bz2')

#### Clean data

##### Process Weather Dataset

In [10]:
# check weather for null columns
weather_df.isnull().sum()

date                         0
latitude                     0
longitude                    0
temperature_2m           30096
relative_humidity_2m     30096
dew_point_2m             30096
pressure_msl             30096
surface_pressure         30096
visibility              363888
wind_speed_10m           30096
wind_speed_80m           30096
wind_speed_100m         363888
wind_speed_120m          30096
wind_speed_180m          30096
wind_direction_10m       30096
wind_direction_80m       30096
wind_direction_100m     363888
wind_direction_120m      30096
wind_direction_180m      30096
temperature_80m          30096
temperature_120m         30096
temperature_180m         30096
type                         0
dtype: int64

In [11]:
# drop unused column
weather_df.drop(columns=['type', 'visibility', 'wind_speed_100m', 
                         'wind_direction_100m'], inplace=True, errors='ignore')
# delete column with null temperature
weather_df.dropna(subset=['temperature_2m'], inplace=True)

# convert column date to datetime
weather_df['date'] = pd.to_datetime(weather_df['date'], utc=True, format='ISO8601')

# check weather for null columns
weather_df.isnull().sum()

date                    0
latitude                0
longitude               0
temperature_2m          0
relative_humidity_2m    0
dew_point_2m            0
pressure_msl            0
surface_pressure        0
wind_speed_10m          0
wind_speed_80m          0
wind_speed_120m         0
wind_speed_180m         0
wind_direction_10m      0
wind_direction_80m      0
wind_direction_120m     0
wind_direction_180m     0
temperature_80m         0
temperature_120m        0
temperature_180m        0
dtype: int64

##### Process Air Quality Dataset

In [13]:
# check air quality for null columns
air_quality_df.isnull().sum()

date                      0
latitude                  0
longitude                 0
pm10                      0
pm2_5                     0
carbon_monoxide           0
carbon_dioxide      3020544
nitrogen_dioxide          0
sulphur_dioxide           0
ozone                     0
dust                      0
uv_index                  0
type                      0
dtype: int64

In [14]:
# drop unused column
air_quality_df.drop(columns=['type','carbon_dioxide'], inplace=True, errors='ignore')

# convert column date to datetime
air_quality_df['date'] = pd.to_datetime(air_quality_df['date'], utc=True, format='ISO8601')

In [15]:
# check air quality for null columns
air_quality_df.isnull().sum()

date                0
latitude            0
longitude           0
pm10                0
pm2_5               0
carbon_monoxide     0
nitrogen_dioxide    0
sulphur_dioxide     0
ozone               0
dust                0
uv_index            0
dtype: int64

In [16]:
# review data date range for weather
weather_df['date'].describe()

count                                6076656
mean     2024-10-26 17:00:00.000001024+00:00
min                2024-09-10 11:00:00+00:00
25%                2024-10-03 14:00:00+00:00
50%                2024-10-26 17:00:00+00:00
75%                2024-11-18 20:00:00+00:00
max                2024-12-11 23:00:00+00:00
Name: date, dtype: object

In [17]:
# review data date range for air quality
air_quality_df['date'].describe()

count                                6106752
mean     2024-10-26 11:29:59.999999232+00:00
min                2024-09-10 00:00:00+00:00
25%                2024-10-03 05:45:00+00:00
50%                2024-10-26 11:30:00+00:00
75%                2024-11-18 17:15:00+00:00
max                2024-12-11 23:00:00+00:00
Name: date, dtype: object

In [18]:
# filter data date range to the same as both
weather_df = weather_df.loc[(weather_df['date'] >= '2024-09-11 11:00:00+00:00') & \
                                    (weather_df['date'] <= '2024-12-11 11:00:00+00:00')]

In [19]:
# filter air quality data date range as same as
air_quality_df = air_quality_df.loc[(air_quality_df['date'] >= '2024-09-11 11:00:00+00:00') & \
                                    (air_quality_df['date'] <= '2024-12-11 11:00:00+00:00')]

In [20]:
weather_df.set_index(['date','latitude','longitude'], inplace=True)
air_quality_df.set_index(['date','latitude','longitude'], inplace=True)

# combine weather and air quality
env_df = pd.concat([weather_df, air_quality_df], axis=1) \
        .reset_index()

In [21]:
# show sample combined data
env_df.head()

,date,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,pressure_msl,surface_pressure,wind_speed_10m,wind_speed_80m,...,temperature_120m,temperature_180m,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,dust,uv_index
0,2024-09-11 11:00:00+00:00,13.3,97.10,27.75,85.0,24.996260,1007.6,1007.6,20.172853,22.768396,...,26.1,25.3,28.8,14.3,107.0,0.1,0.1,44.0,4.0,0.05
1,2024-09-11 11:00:00+00:00,13.3,97.25,27.55,87.0,25.190360,1007.6,1007.6,19.083395,20.883102,...,25.8,25.0,28.8,14.3,107.0,0.1,0.1,44.0,4.0,0.05
2,2024-09-11 11:00:00+00:00,13.3,97.40,27.45,87.0,25.092102,1007.7,1007.7,18.374111,20.172853,...,25.8,24.9,29.1,14.5,106.0,0.1,0.1,44.0,4.0,0.05
3,2024-09-11 11:00:00+00:00,13.3,97.55,27.45,88.0,25.284054,1007.7,1007.7,17.786331,19.586119,...,25.7,24.9,29.1,14.5,106.0,0.1,0.1,44.0,4.0,0.05
4,2024-09-11 11:00:00+00:00,13.3,97.70,27.40,88.0,25.234846,1007.7,1007.7,16.489416,17.786331,...,25.7,24.8,29.1,14.5,106.0,0.1,0.1,44.0,4.0,0.05


In [22]:
# save to parquet format
env_df.to_parquet('../data/processed/air_quality_data-20240911_20241211.parquet')

---

##### Process Satellite Hotspot Dataset

In [25]:
# list all data files
files = list(os.walk(hotspot_dir))
files = files.pop()

# create new empty dataframe
hotspot_df = pd.DataFrame()

# process all files
for file in files[2]:
    if 'nrt' not in file:
        continue
    print(f'processing file {file}')

    # read data from csv, and spectific data type to acq_time to prevent truncate leading zero
    df = pd.read_csv(f'{files[0]}/{file}', dtype={'acq_time': 'object'})
    
    # construct a new datetime column by merging the existing date and time columns.
    df['date'] = df['acq_date'] + ' ' + df['acq_time']
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H%M', utc=True)
    df.drop(['acq_date','acq_time'], inplace=True, axis=1)
    
    # concat all dataframe together
    hotspot_df = pd.concat([hotspot_df, df])

processing file fire_nrt_J1V-C2_557681.csv
processing file fire_nrt_J2V-C2_557682.csv
processing file fire_nrt_M-C61_557680.csv
processing file fire_nrt_SV-C2_557683.csv


In [26]:
# check for duplicates rows
hotspot_df[hotspot_df.duplicated(subset=['date','latitude','longitude','satellite'])]

,latitude,longitude,brightness,scan,track,satellite,instrument,confidence,version,bright_t31,frp,daynight,date


In [27]:
# check for null values
hotspot_df.isnull().sum()

latitude      0
longitude     0
brightness    0
scan          0
track         0
satellite     0
instrument    0
confidence    0
version       0
bright_t31    0
frp           0
daynight      0
date          0
dtype: int64

In [28]:
# review data type
hotspot_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333357 entries, 0 to 5950
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   latitude    333357 non-null  float64            
 1   longitude   333357 non-null  float64            
 2   brightness  333357 non-null  float64            
 3   scan        333357 non-null  float64            
 4   track       333357 non-null  float64            
 5   satellite   333357 non-null  object             
 6   instrument  333357 non-null  object             
 7   confidence  333357 non-null  object             
 8   version     333357 non-null  object             
 9   bright_t31  333357 non-null  float64            
 10  frp         333357 non-null  float64            
 11  daynight    333357 non-null  object             
 12  date        333357 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(7), object(5)
memory usage: 35.6+ MB


In [29]:
# convert confidence values to strings to avoid export errors
hotspot_df['confidence'] = hotspot_df['confidence'].astype(str)

In [30]:
hotspot_df = hotspot_df.loc[(hotspot_df['date'] >= '2024-09-11 00:00:00+00:00') & \
                            (hotspot_df['date'] <= '2024-12-11 23:59:59+00:00')]

In [31]:
# set index and export
hotspot_df.set_index('date').to_parquet('../data/processed/sat_hotspot-20240911_20241211.parquet')

#### Merge environment and satellite hotspot data

In [34]:
# set coordinate degree step
loc_step = 0.5
round_div = 1 / loc_step
loc_env_df = env_df.set_index(['date','latitude','longitude'], drop=False)
loc_env_df = loc_env_df[(loc_env_df['date']>=pd.to_datetime('2024-11-01 00:00:00+00:00')) & \
                        (loc_env_df['date']<=pd.to_datetime('2024-11-30 23:59:59+00:00'))]

# set index
loc_hotspot_df = hotspot_df.set_index(['date','latitude','longitude'], drop=False)
# round to loc_step
loc_env_df['latitude'] = (loc_env_df['latitude'] * round_div).round() / round_div
loc_env_df['longitude'] = (loc_env_df['longitude'] * round_div).round() / round_div

# set index
loc_hotspot_df = loc_hotspot_df.drop(columns=['date','latitude','longitude']).reset_index()

# round to loc_step
loc_hotspot_df['latitude'] = (loc_hotspot_df['latitude'] * round_div).round() / round_div
loc_hotspot_df['longitude'] = (loc_hotspot_df['longitude'] * round_div).round() / round_div
# round date to 12 hours
loc_hotspot_df['date'] = loc_hotspot_df['date'].dt.round('12h')
loc_hotspot_df.set_index(['date','latitude','longitude'], drop=False, inplace=True)

In [35]:
loc_hotspot_df['hotspot_num'] = loc_hotspot_df.drop(columns=['date','latitude','longitude']) \
                    .groupby(by=['date','latitude','longitude']).size()

In [36]:
env_air_df = pd.merge(left=loc_env_df.set_index(['date','latitude','longitude']), 
                      right=loc_hotspot_df.set_index(['date','latitude','longitude'])[['hotspot_num']],
                      left_index=True, right_index=True, how='left')
env_air_df['hotspot_num'] = env_air_df['hotspot_num'].ffill()
avg_env_air_df = env_air_df.reset_index().groupby(['date','latitude','longitude']).mean()

In [37]:
# save to parquet format
avg_env_air_df.to_parquet('../data/processed/air_quality_hotspot_data-20241101_20241130.parquet')